# Bricklayer Robot

In [1]:
# Extensions: magic to avoid re-loading modified .py modules all the time
%load_ext autoreload
%autoreload 2

# Bookkeeping
import sys
import shutil
import os as os
import matplotlib.pyplot as plt

# Custom project classes and functions
from iiwa import *
from Trajectory import *
from TrajectoryBuilder import *
from Visualizer import *
from BrickPlanner import *

# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

# Start the visualizer.
meshcat = StartMeshcat()

/home/nikita/env/lib/python3.10/site-packages/mpld3/mplexporter/renderers/base.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
INFO:drake:Meshcat listening for connections at http://localhost:7000


Working in directory  /home/nikita/MIT_6_4212/project


In [2]:
# System configs
# Brick configuration
kNumberOfBricks =  150 # @param
# Brick dimensions, must be consistent with the geometry in models/real_brick.sdf
kBrickGeom = np.array([0.075, 0.1, 0.05]) # @param

# Brick warehouse config
kWhLocation = np.array([2.5, 2.5, 0.15]) # location of the warehouse (corner), # @param
kWhSize = np.array([5, 5, 10]) # size of the warehouse grid in cells, # @param
if (kWhSize.prod() < kNumberOfBricks):
    assert False, "The warehouse is too small to fit all bricks"

# Brick source
X_WBrickSource = RigidTransform(np.array([0.6, 0.0, kBrickGeom[2]/2])) # @param

In [3]:
# Instantiate the robot (TODO: remove it from here)
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom)

/home/nikita/MIT_6_4212/project/iiwa.py:117: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(context)


In [4]:
# Generate bricks
floorplan = """
|
|
|
"""

floorplan_0 = """
|- - - - - |
|          |
|          |
|          |
|- - - - - |
"""

floorplan_1 = """
|- - - - - |
|          |
|- - -     |
      |    |
      |    |
|- - -     |
|          |
|- - - - - |
"""

floorplan_2 = """
|- - - - - - - - |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|- - - - - - - - |
"""

kHeight = 3 # @param
kGap = 0.005 # @param
planer = Planner(floorplan_2, kBrickGeom, kGap, kHeight)
#planer.shift(0.35, -0.4)
#planer.shift(-0.3, 0.4)
planer.shift(-0.55, -0.45)
total_bricks_needed = planer.get_total_bricks_n()
if (total_bricks_needed > kNumberOfBricks):
    assert False, "There is not enough bricks to cover the building"
print("Bricks planned, total bricked needed: ", total_bricks_needed)

# Visualize floorplan
vis = Visualizer(meshcat, kBrickGeom)
vis.clear_vis()
vis.visualize_plan(np.array(planer.get_brick_poses()))

Parsing floorplan:  
|- - - - - - - - |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|                |
|- - - - - - - - |

Bricks planned, total bricked needed:  108


In [200]:
# Build trajectories
# Build initial trajectory and get q0 in the calibration point
trj_builder = TrajectoryBuilder(ilyich, X_WBrickSource)
trj_builder.gen_initial_traj()
#err = trj_builder.solve_IK(np.array([ 0., 0.6, 0., 0.0, 0.0, 1, 0.0, 0., 0.]))
res, q0 = trj_builder.solve_IK(np.array([ 0., 0.0, 0., 0.0, 0.0, 0.0, 0.0, 0., 0.]))
assert res == True, "Failed to solve IK for the initial trajectory"

print("q0 in calibration point is ", q0)

context = ilyich.CreateDefaultContext()
plant_context = ilyich.diagram.GetMutableSubsystemContext(ilyich.plant, context)

c_opt = IKOptimizationController()

covered_bricks = []
failed_bricks = []
for i in range(0, 3):#total_bricks_needed):
    # Set points
    b = planer.get_brick_poses()[i]
    X_WBrickTarget = RigidTransform(RotationMatrix(), b[:3])

    # Generate trajectory for brick
    trj_builder_for_brick = TrajectoryBuilder(ilyich, X_WBrickSource)
    # Grab
    trj_builder_for_brick.gen_grab_brick_traj(i)
    res, q = trj_builder_for_brick.solve_IK(q0)
    if res == False:
        print("Failed to solve IK for brick: ", i)
        failed_bricks.append(i)
        continue

    # Move to place
    q, (move_t, move_step) = trj_builder_for_brick.gen_move_to_place_traj(q, X_WBrickTarget, i)

    # Place
    trj_builder_for_brick.gen_place_brick_traj(X_WBrickTarget, b[3], i)
    res, q = trj_builder_for_brick.solve_IK(q0)
    if res == False:
        print("Failed to solve IK for brick: ", i)
        failed_bricks.append(i)
        continue

    # Move back
    q, (move_t, move_step) = trj_builder_for_brick.move_back(q, X_WBrickTarget, i)

    # Return
    #if not bypass_P == P2:
    #    trj_builder_for_brick.gen_move_to_place_traj(
    #        RigidTransform(np.array([bypass_p[0], bypass_p[1], X_WBrickTarget.translation()[2]])),
    #        i)
    #trj_builder_for_brick.gen_return_to_source_traj(i)
    ## Solve IK for the trajectories
    #res, q = trj_builder_for_brick.solve_IK(q0)
    #if res == False:
    #    # Failed to solve for this bypass, try another one
    #    success = False
    #    continue
    #else:
    #    # Solved for this bypass, accept it
    #    success = True
    #    covered_bricks.append(i)
    #    trj_builder.merge(trj_builder_for_brick)
    #    break
    
    covered_bricks.append(i)
    trj_builder.merge(trj_builder_for_brick)

    #
    #assert False, "Done"

    # Failed bricks 
    #if success == False:
    ##    print("Failed to solve IK for brick: ", i)
    #    failed_bricks.append(i)

# Print stat
print("Covered bricks: ", covered_bricks)
print("Failed bricks:", failed_bricks)

# Speed-up a bit
trj_builder.get_trajectories().slow_down(0.5)

# Make trajectories for iiwa with IK
traj, finger_traj = trj_builder.get_trajectories().form_iiwa_traj_q()

q0 in calibration point is  [ 0.14557608  0.67413625 -0.22359343 -1.39006593  0.15704599  1.09444912
  1.46900663  0.          0.        ]
Solving...
Failed to construct Move trajectory


AssertionError: 

In [186]:
vis.clear_vis()
points = trj_builder.get_trajectory_points_for_brick(1)
vis.visualize_traj(points, 1)

In [162]:
print(trj_builder.get_trajectories().get_breakpoints())

[1.000005, 8.20000999999999, 14.90001499999998]


In [191]:
# Visualize coverage
vis = Visualizer(meshcat, kBrickGeom)
vis.clear_vis()
vis.visualize_coverage(planer.get_brick_poses()[covered_bricks], planer.get_brick_poses()[failed_bricks])

# Visualize all trajectories
for b_id in covered_bricks:
    points = trj_builder.get_trajectory_points_for_brick(b_id)
    vis.visualize_traj(points, b_id)

In [187]:
# Create robot
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom, [traj, finger_traj])

# Create default context
context = ilyich.CreateDefaultContext()

# Put all bricks in the warehouse
ilyich.put_bricks_in_warehouse(context, kWhLocation, kWhSize, kNumberOfBricks)

In [188]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = ilyich.work(None, context, sim_duration=trj_builder.get_trajectories().get_breakpoints()[0])
q0 = ilyich.get_q0(ctx)
print("calibration q0= ", q0)

# Continue for all bricks
for i, b_id in enumerate(covered_bricks):
    # Show trajectory
    points = trj_builder.get_trajectory_points_for_brick(b_id)
    vis.clear_traj()
    vis.visualize_traj(points)

    # Teleport the next new brick from the warehouse and unlock it
    ilyich.move_brick(context, X_WBrickSource, i)
    ilyich.unlock_brick(context, i)

    # Manipulate the brick to target
    sim, ctx = ilyich.work(sim, ctx, sim_duration=trj_builder.get_trajectories().get_breakpoints()[i+1])

    # Lock brick
    ilyich.lock_brick(context, i)

calibration q0=  [ 0.14825553  0.66855888 -0.22518602 -1.39959155  0.15436727  1.08739172
  1.47879735]


IndexError: list index out of range